<a href="https://colab.research.google.com/github/chj1210/Crime_Investigation/blob/main/transaction_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Faker

import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# 初始化 Faker，用於生成擬真數據
fake = Faker('zh_TW') # 使用中文 (台灣) 語系，讓地點和名稱更貼近
Faker.seed(0) # 設定種子，確保每次執行結果一致

# 定義交易類型
transaction_types = ['轉帳', '消費', '存款', '提款', '繳費']

# 模擬生成一筆交易數據
def generate_transaction(is_fraud=False):
    """
    生成一筆單獨的交易數據。
    is_fraud: 如果為 True，則生成一筆可能被視為「詐欺」的交易 (例如，金額較大，時間較晚)。
              這是一個簡單的模擬，實際詐欺模式複雜得多。
    """
    timestamp = fake.date_time_between(start_date='-1y', end_date='now')

    # 模擬地點 (可以更細緻，這裡簡化為城市)
    location = fake.city()

    # 模擬帳戶號碼 (簡單的數字串)
    account_number = fake.bban() # 基本銀行帳號號碼

    # 交易金額
    if is_fraud:
        amount = round(random.uniform(50000, 500000), 2) # 較大金額
        # 模擬詐欺交易可能發生在非正常工作時間
        if random.random() < 0.7: # 70% 機率在深夜或凌晨
            hour = random.choice(list(range(22, 24)) + list(range(0, 6)))
            timestamp = timestamp.replace(hour=hour, minute=random.randint(0, 59), second=random.randint(0, 59))
        transaction_type = random.choice(['轉帳', '消費']) # 詐欺可能集中在特定類型
    else:
        amount = round(random.uniform(100, 10000), 2) # 一般金額
        transaction_type = random.choice(transaction_types)

    return {
        '交易ID': fake.uuid4(),
        '時間': timestamp,
        '帳戶': account_number,
        '金額': amount,
        '交易類型': transaction_type,
        '地點': location,
        '是否詐欺': is_fraud # 用於標示少數類別
    }

# 生成多筆交易數據
def generate_dataset(num_normal_transactions, num_fraud_transactions):
    """
    生成包含正常交易和詐欺交易的數據集。
    num_normal_transactions: 正常交易的數量
    num_fraud_transactions: 詐欺交易的數量 (作為少數類別)
    """
    data = []
    print(f"正在生成 {num_normal_transactions} 筆正常交易...")
    for _ in range(num_normal_transactions):
        data.append(generate_transaction(is_fraud=False))

    print(f"正在生成 {num_fraud_transactions} 筆詐欺交易 (少數類別)...")
    for _ in range(num_fraud_transactions):
        data.append(generate_transaction(is_fraud=True))

    df = pd.DataFrame(data)
    # 打亂數據以避免順序偏差
    df = df.sample(frac=1).reset_index(drop=True)
    return df

# 範例使用：
# 生成 10000 筆正常交易和 50 筆詐欺交易 (模擬數據不平衡)
num_normal = 10000
num_fraud = 50
df_transactions = generate_dataset(num_normal, num_fraud)

print("\n--- 合成金融交易數據範例 (前 5 筆) ---")
print(df_transactions.head())

print("\n--- 數據集統計資訊 ---")
print(df_transactions['是否詐欺'].value_counts())
print(f"數據集總筆數：{len(df_transactions)} 筆")

# 您也可以將數據保存到 CSV 文件
# df_transactions.to_csv('synthetic_financial_transactions.csv', index=False, encoding='utf-8-sig')
# print("\n數據已保存到 synthetic_financial_transactions.csv")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.3 MB/s eta 0:00:00
正在生成 10000 筆正常交易...
正在生成 50 筆詐欺交易 (少數類別)...

--- 合成金融交易數據範例 (前 5 筆) ---
                                   交易ID                         時間  \
0  c9e1920f-d316-4c70-9aa0-f9bd908d23ff 2025-02-14 20:45:34.287832   
1  8591b806-e506-4c00-9553-2e7e22dd31ed 2024-09-22 19:17:17.341475   
2  cccf7520-9e59-4ec4-9f8c-3e274fcb729e 2024-11-15 03:49:58.583905   
3  f7e735f4-d018-45be-bb3c-dd024cefa0c9 2025-03-02 12:05:52.065910   
4  02fd6ed3-0cfa-4aa0-a91f-42792889e381 2025-01-26 19:12:34.925972   

                   帳戶       金額 交易類型   地點   是否詐欺  
0  TTXA76814815986452  6555.13   繳費  雲林縣  False  
1  WVIN29616807119173  5470.47   轉帳  朴子市  False  
2  VWGD08121031112138  9114.16   存款  屏東市  False  
3  HRQK63215199967771  1010.18   繳費   北斗  False  
4  HXBN57427185352897  6601.91   存款   新營  False  

--- 數據集統計資訊 ---
是否詐欺
False    10000
True        50
Name: count, dtype: int64
數據集總筆數：10050 筆
